In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hithadhoo,MV,2022-06-07 18:11:15,-0.6000,73.0833,83.50,66,97,4.43
1,1,Qaanaaq,GL,2022-06-07 18:12:54,77.4840,-69.3632,31.37,85,86,2.24
2,2,Jamestown,US,2022-06-07 18:07:55,42.0970,-79.2353,67.59,75,100,8.05
3,3,Dorado,PR,2022-06-07 18:11:06,18.4588,-66.2677,92.71,74,40,16.11
4,4,Ushuaia,AR,2022-06-07 18:11:46,-54.8000,-68.3000,44.22,86,63,7.81


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [27]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [28]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [30]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [31]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [32]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [34]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hithadhoo,MV,2022-06-07 18:11:15,-0.6000,73.0833,83.50,66,97,4.43
8,8,Rey Bouba,CM,2022-06-07 18:12:57,8.6724,14.1786,88.75,41,18,4.52
12,12,Sechura,PE,2022-06-07 18:12:58,-5.5569,-80.8222,75.09,59,57,9.64
14,14,Hobyo,SO,2022-06-07 18:12:59,5.3505,48.5268,81.23,71,88,21.34
21,21,Avarua,CK,2022-06-07 18:11:13,-21.2078,-159.7750,77.05,78,75,11.50
22,22,Timizart,DZ,2022-06-07 18:13:01,36.8000,4.2667,75.87,58,59,2.80
24,24,Albany,US,2022-06-07 18:07:16,42.6001,-73.9662,79.52,54,98,3.00
26,26,Kashi,CN,2022-06-07 18:13:03,39.4547,75.9797,77.52,24,25,6.55
30,30,Atuona,PF,2022-06-07 18:13:05,-9.8000,-139.0333,77.76,71,8,15.57
33,33,Victoria,HK,2022-06-07 18:09:31,22.2855,114.1577,80.65,90,100,3.00


In [35]:
preferred_cities_df.count()

City_ID       182
City          182
Country       182
Date          182
Lat           182
Lng           182
Max Temp      182
Humidity      182
Cloudiness    182
Wind Speed    182
dtype: int64

In [36]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,83.50,-0.6000,73.0833,
8,Rey Bouba,CM,88.75,8.6724,14.1786,
12,Sechura,PE,75.09,-5.5569,-80.8222,
14,Hobyo,SO,81.23,5.3505,48.5268,
21,Avarua,CK,77.05,-21.2078,-159.7750,
22,Timizart,DZ,75.87,36.8000,4.2667,
24,Albany,US,79.52,42.6001,-73.9662,
26,Kashi,CN,77.52,39.4547,75.9797,
30,Atuona,PF,77.76,-9.8000,-139.0333,
33,Victoria,HK,80.65,22.2855,114.1577,


In [37]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [38]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [39]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEAJz5Qqafm63T-hJ5wuC83wMs8VM0EiCoQmiMz5Z1JkpfYH3I9bPKauLrXGz8yBKpLT5bHc1dDRLBB5lleFFhal5qPeK74EvESqleTNygCA8iID7Dy2OlrldP6qNn1miUNb1ym3gW_YEV67yLJiJSe6-poryxiDbPDl13mKqV-RMrO2tr9VRYHkfyGutZQ5Xv4zIXdq_WqVFPGStmMOFEhO8lBHvikMJD1iYDQPApRmu6SHMaLahGGEztN0g7b8NOlbndfyhHyqI9v9NkskjqzP0ds5EMhqQGzZSRdspmJwW_mYwWuwSsOrLT_2t7n-9RdH7zQbPgtZ270R5-5xXzOuu791oTjg-Te86rLQKo_iS1jYeY_PAeYtvWh9aiNs25tW0viNMj1KvS2kok8SI5nNxj9tWkjOgA74og8rRyApJrLON3pdftC-',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [51]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [52]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,83.50,-0.6000,73.0833,Scoop Guest House
8,Rey Bouba,CM,88.75,8.6724,14.1786,
12,Sechura,PE,75.09,-5.5569,-80.8222,Hospedaje Costa Bella
14,Hobyo,SO,81.23,5.3505,48.5268,Iftin hotel hobyo
21,Avarua,CK,77.05,-21.2078,-159.7750,Paradise Inn


In [53]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [55]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [56]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [57]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))